In [1]:
from langchain_google_genai import GoogleGenerativeAI

import os
from dotenv import load_dotenv
load_dotenv() 

#llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=os.environ["GOOGLE_API_KEY"])
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=os.environ["GOOGLE_API_KEY"])

C:\Users\bhave\anaconda3\Lib\site-packages\langchain_google_genai\__init__.py:59: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_google_genai.chat_models import ChatGoogleGenerativeAI


TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [2]:
pip install --upgrade pydantic langchain langchain-google-genai


  Obtaining dependency information for langchain-google-genai from https://files.pythonhosted.org/packages/56/22/c3617368229ec45dd8a9c19fe2b9cdfb35cdc8460a19d92331358aedb2c2/langchain_google_genai-2.0.7-py3-none-any.whl.metadata
  Obtaining dependency information for filetype<2.0.0,>=1.2.0 from https://files.pythonhosted.org/packages/18/79/1b8fa1bb3568781e84c9200f951c735f3f157429f44be0495da55894d620/filetype-1.2.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-generativeai<0.9.0,>=0.8.0 from https://files.pythonhosted.org/packages/e9/2f/b5c1d62e94409ed98d5425e83b8e6d3dd475b611be272f561b1a545d273a/google_generativeai-0.8.3-py3-none-any.whl.metadata
  Obtaining dependency information for google-ai-generativelanguage==0.6.10 from https://files.pythonhosted.org/packages/af/6d/db99a295f9caf027bbdd90c41e6ea650a7468392a0e8713719e7abc5f647/google_ai_generativelanguage-0.6.10-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/41.3 kB ? eta -:-

In [1]:
from langchain_google_genai import GoogleGenerativeAI

import os
from dotenv import load_dotenv
load_dotenv() 

#llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=os.environ["GOOGLE_API_KEY"])
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=os.environ["GOOGLE_API_KEY"])

In [2]:
print(llm("write few lines on Generative AI"))

C:\Users\bhave\AppData\Local\Temp\ipykernel_12432\266188152.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm("write few lines on Generative AI"))


Generative AI, a subset of artificial intelligence, empowers computers to create new content, mimicking human creativity. It can generate text, images, music, videos, and more. Unlike traditional AI, which focuses on analyzing and making predictions, generative AI uses deep learning to learn patterns and create novel outputs. This technology opens up endless possibilities for enhancing human capabilities, from automating creative tasks to fostering innovation in various industries.


### Connect with databse and ask basic questions

In [3]:
#from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

from langchain.utilities import SQLDatabase

In [4]:
db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "retail_sales_db"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)


print(db.table_info)


CREATE TABLE sales_tb (
	`TransactionID` INTEGER, 
	`Date` DATE, 
	`CustomerID` VARCHAR(10), 
	`Gender` VARCHAR(10), 
	`Age` INTEGER, 
	`ProductCategory` VARCHAR(50), 
	`Quantity` INTEGER, 
	`PriceperUnit` DECIMAL(10, 2), 
	`TotalAmount` DECIMAL(10, 2)
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from sales_tb table:
TransactionID	Date	CustomerID	Gender	Age	ProductCategory	Quantity	PriceperUnit	TotalAmount
1	2023-11-24	CUST001	Male	34	Beauty	3	50.00	150.00
2	2023-02-27	CUST002	Female	26	Clothing	2	500.00	1000.00
3	2023-01-13	CUST003	Male	50	Electronics	1	30.00	30.00
*/


In [5]:
#Convert question to SQL query
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many customers are there"})
response

'```sql\nSELECT DISTINCT `CustomerID` \nFROM `sales_tb`\nLIMIT 5;\n```'

In [6]:
cleaned_query = response.strip('```sql\n').strip('\n```')
print(cleaned_query)

SELECT DISTINCT `CustomerID` 
FROM `sales_tb`
LIMIT 5;


In [7]:
# Execute the cleaned query
result = db.run(cleaned_query)
print(result)

[('CUST001',), ('CUST002',), ('CUST003',), ('CUST004',), ('CUST005',)]


In [8]:
chain = create_sql_query_chain(llm, db)
def execute_query(question):
    try:
        # Generate SQL query from question
        response = chain.invoke({"question": question})
        print(response)
        print("###################################################")
        # Strip the formatting markers from the response
        cleaned_query = response.strip('```sql\n').strip('\n```')
        print(cleaned_query)
        print("###################################################")        
        # Execute the cleaned query
        result = db.run(cleaned_query)
        print("###################################################")        
        # Display the result
        print(result)
    except ProgrammingError as e:
        print(f"An error occurred: {e}")

In [9]:
q1 = "How many unique customers are there for each product category"
execute_query(q1)

```sql
SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS `UniqueCustomers` 
FROM sales_tb 
GROUP BY `ProductCategory` 
ORDER BY `UniqueCustomers` DESC 
LIMIT 5;
```
###################################################
SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS `UniqueCustomers` 
FROM sales_tb 
GROUP BY `ProductCategory` 
ORDER BY `UniqueCustomers` DESC 
LIMIT 5;
###################################################
###################################################
[('Clothing', 13), ('Beauty', 8), ('Electronics', 8)]


In [10]:
q2 = "Calculate total sales amount per product category:"
execute_query(q2)

```sql
SELECT 
    `ProductCategory`, 
    SUM(`TotalAmount`) AS `TotalSalesAmount`
FROM 
    `sales_tb`
GROUP BY 
    `ProductCategory`
ORDER BY 
    `TotalSalesAmount` DESC
LIMIT 
    5;
```
###################################################
SELECT 
    `ProductCategory`, 
    SUM(`TotalAmount`) AS `TotalSalesAmount`
FROM 
    `sales_tb`
GROUP BY 
    `ProductCategory`
ORDER BY 
    `TotalSalesAmount` DESC
LIMIT 
    5;
###################################################
###################################################
[('Electronics', Decimal('5310.00')), ('Clothing', Decimal('5040.00')), ('Beauty', Decimal('1455.00'))]


In [11]:
q3 = "calculates the average age of customers grouped by gender."
execute_query(q3)

```sql
SELECT `Gender`, AVG(`Age`) AS `Average Age`
FROM sales_tb
GROUP BY `Gender`
ORDER BY `Average Age` DESC
LIMIT 5;
```
###################################################
SELECT `Gender`, AVG(`Age`) AS `Average Age`
FROM sales_tb
GROUP BY `Gender`
ORDER BY `Average Age` DESC
LIMIT 5;
###################################################
###################################################
[('Female', Decimal('43.3333')), ('Male', Decimal('35.2143'))]


In [12]:
q4 = "identify the top spending customers based on their total amount spent."
execute_query(q4)

```sql
SELECT 
    `CustomerID`, 
    SUM(`TotalAmount`) AS `TotalAmountSpent`
FROM 
    `sales_tb`
GROUP BY 
    `CustomerID`
ORDER BY 
    `TotalAmountSpent` DESC
LIMIT 
    5;
```
###################################################
SELECT 
    `CustomerID`, 
    SUM(`TotalAmount`) AS `TotalAmountSpent`
FROM 
    `sales_tb`
GROUP BY 
    `CustomerID`
ORDER BY 
    `TotalAmountSpent` DESC
LIMIT 
    5;
###################################################
###################################################
[('CUST015', Decimal('2000.00')), ('CUST013', Decimal('1500.00')), ('CUST016', Decimal('1500.00')), ('CUST002', Decimal('1000.00')), ('CUST026', Decimal('1000.00'))]


In [13]:
q5 = "counts the number of transactions made each month."
execute_query(q5)

```sql
SELECT
  strftime('%Y-%m', `Date`) AS sales_month,
  COUNT(*) AS transaction_count
FROM sales_tb
GROUP BY sales_month
ORDER BY transaction_count DESC
LIMIT 5;
```
###################################################
SELECT
  strftime('%Y-%m', `Date`) AS sales_month,
  COUNT(*) AS transaction_count
FROM sales_tb
GROUP BY sales_month
ORDER BY transaction_count DESC
LIMIT 5;
###################################################


NameError: name 'ProgrammingError' is not defined

In [14]:
q6 = "calculates the total sales amount and average price per unit for each product category."
execute_query(q6)

```sql
SELECT 
  `ProductCategory`,
  SUM(`TotalAmount`) AS `TotalSalesAmount`,
  AVG(`PriceperUnit`) AS `AveragePricePerUnit`
FROM `sales_tb`
GROUP BY 
  `ProductCategory`
ORDER BY 
  `TotalSalesAmount` DESC,
  `AveragePricePerUnit` DESC
LIMIT 5;
```
###################################################
SELECT 
  `ProductCategory`,
  SUM(`TotalAmount`) AS `TotalSalesAmount`,
  AVG(`PriceperUnit`) AS `AveragePricePerUnit`
FROM `sales_tb`
GROUP BY 
  `ProductCategory`
ORDER BY 
  `TotalSalesAmount` DESC,
  `AveragePricePerUnit` DESC
LIMIT 5;
###################################################
###################################################
[('Electronics', Decimal('5310.00'), Decimal('238.750000')), ('Clothing', Decimal('5040.00'), Decimal('183.461538')), ('Beauty', Decimal('1455.00'), Decimal('153.750000'))]
